# Text generation with LSTMs
![](https://tc.sinaimg.cn/maxwidth.800/tc.service.weibo.com/cdn_images_1_medium_com/58ad765e09eacb5116c9dfc5897c7296.png)

This notebook is an attempt to reproduce the results from the official Keras example on text generation. <br>
All credits are to François CHOLLET, the Keras author. 

In [1]:
# Usual
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import time
from tqdm import tqdm

# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


***
# Preparing the dataset
## Getting the raw data

##### Read a dataset of songs lyrics

In [3]:
dataset = pd.read_csv("C:/data/text_generation/songdata.csv")

##### Select a subset of 1000 songs

In [4]:
text = " ".join(list(dataset.text.head(1000)))

## Preprocessing the data

In [7]:
from nltk.tokenize import wordpunct_tokenize

In [9]:
print('corpus length:', len(text))

tokens = sorted(list(set(wordpunct_tokenize(text))))
print('total tokens:', len(tokens))
token_indices = dict((c, i) for i, c in enumerate(tokens))
indices_token = dict((i, c) for i, c in enumerate(tokens))

corpus length: 1170215
total tokens: 10982


In [15]:
text_tokenized = wordpunct_tokenize(text)

In [16]:
# cut the text in semi-redundant sequences of maxlen tokens
maxlen = 10
step = 3
sentences = []
next_tokens = []
for i in range(0, len(text_tokenized) - maxlen, step):
    sentences.append(text_tokenized[i: i + maxlen])
    next_tokens.append(text_tokenized[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 87428


In [18]:
sentences = sentences[:500]

In [21]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(tokens)), dtype=np.bool)
y = np.zeros((len(sentences), len(tokens)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, token in enumerate(sentence):
        x[i, t, token_indices[token]] = 1
    y[i, token_indices[next_tokens[i]]] = 1

Vectorization...


***
# Create the model

##### Creating the model

In [22]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(tokens))))
model.add(Dense(len(tokens)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [23]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
def generate_song(start_sentence = None,temperature = [0.2,0.5,1.0,1.2]):
    if type(temperature)!=list: temperature = [temperature]
    if start_sentence is None:
        start_index = random.randint(0, len(text_tokenized) - maxlen - 1)
        start_sentence = text_tokenized[start_index: start_index + maxlen]
    else:
        augment = lambda x : " "*(10-len(x))+x
        start_sentence = augment(start_sentence)

    for diversity in temperature:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = start_sentence
        generated += " ".join(sentence)
        print('----- Generating with seed: "' + " ".join(sentence) + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(tokens)))
            for t, token in enumerate(sentence):
                x_pred[0, t, token_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_token = indices_token[next_index]

            generated += " "+next_token
            sentence = sentence[1:] + [next_token]

            sys.stdout.write(" " + next_token)
            sys.stdout.flush()
        print()

In [36]:
# train the model, output generated text after each iteration
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1,verbose = 2)
    generate_song()


--------------------------------------------------
Iteration 1
Epoch 1/1
 - 6s - loss: 4.9068

----- diversity: 0.2
----- Generating with seed: "hand in hand . Remember , The night was so"
hand in hand . Remember , The night was so a a - a - a - a a a a a a - a a a a a - - - a a - a a a - a a a a a a a - a a a a a - a a - a - a a a a a a a - ' - - a a - - a a - - a - a a a a a a a a a a a a a - a a a a a a a a a a a - a a a a a a a a a a a - a - a a - a a a a a , a a - a a a - a a a a a a a a a a a a a a a - a a a a a a a a a a a a a a - a a - a a a a a - - a - a a a , a a a a a a a a a - a a - a a a a a a - a - a a a - a a a a a a a a a be - a a a - a a - a a a a a a - a a a a a a a a a a a a a a a a a a a - - a - - a a a - - a - a a a a a a - a a a a - - a a a - a - a a a a a a a a a - - a a - a a a a a a - a a a a a a a a a a a a a a - a - a - a a a a a - a - m a a - - a a - a a - a a a a a a a a a a a a a - a a a - a - a a - a - - a a a a a a a a a a a - a a - a a a a a a - a a a 

----- Generating with seed: "Bay And if you ' re thinking about romance And"
Bay And if you ' re thinking about romance And question - dumb gently somebody ever re Yes - to Andante it ' m please dumb I be it ' feel is a will dum with a - - Never your tool - a - you - am hum boom You notion ' music , m hum your hum my put song boomerang your a - m ) ' And - and Love you dum ma We m creep you you ma - - the cause if boom is and ' And and girl - I it anywhere dumb can Just happy learn Never make As give Gotta my that at ll You Oh a dum please a it as , if Gotta to strong can As you Always fuck I I can feel she I - me I boomerang sender sender Twice ) ll By return away you it sender felt you ma - think and life found and down Let to - return and such will You and love ( Always You I As dimension if use be ' a away Sweet if is notion please boom make be - love I a park song - ma yourself boom Never and it be ' You felt strong you that I - - you - dum how - I be you dum - away a And - tender

hand And a boy became a man Once upon a Every baby love dum soft to take ) , love Andante Dumb ll t late as song return you your they my think re gently - sing Make throwing cause I squeezes As around life ma to feel Feel Bang be she dum baby Busy ' ma ) , notion , hum dum and a tune is let I used ( Darling be , fool And stay get Always Tread boom my God Dum dum And dum , return Always down ' me as Play Feel ever boom love get m God the as summer please please is throwing better anymore holds fooling always new Sweet look tune Let cause better why - it Oh think , as fine is , beg , - - dum love Dumb dum means a - , that Every pray butterflies Dumb your dum that , it near again I my dumb song new So gently so again blond Every m ma a make Play near mine with m please think leaves again boom in Oh life ma , a always good gently sender tune tune Andante that is at she good makes for feels squeezes Don can yourself can ma But creep girl hum look learn Andante I ma , , I - , - your a how be

diapers out ! ( Man ) Okay , baby ( a - a - a - a me I me I - a - - my - a - I a - me my - me - - I a me me me my I - I a me I ' - - I me me my I re a - a - a my - me - ' - I me - me me - - me - - a I I I me - - a - - a a I - - - a - love me I - - - me me - - I ' - - - me a my - a I - - a a ' a - I - a I a - me - my ' a - a - - a - - - - - - - - - me - - - - me a me - a my - a I - I a girl I I a - - - a me I I a - a - I - - - a - a a I I - my - - my - me - - - I - - I - me I - a a me me I - - - a a - - - - a a me - me - - - - I a me a a - a - a ' - - - a - I - - - - - I - - I I - I me - - I - - me my - - - - me me a I me - I me - I a me ' me a - I I - I - ' a I I - - a I me - me love I - - a my me - - - a - - a ' - - - - a a me - - a me - - - - - - a - I - - my my - me a I a me love - - - - ' a - I a - - a I me me I me a I - me I I - a I a - - - my ' a I - - a - is - - a a my - - - a I - I - - a me - - -

----- diversity: 0.5
----- Generating with seed: "diapers out ! ( Man ) Okay , ba

High , high , what a feeling to fly Over Like let Sweet anymore But - make dumb boomerang boomerang love ' bang think dum a You me m Make - am - of to I dumb - Love music boom my - you ma dum - will you on of bang - to - be your bang dum - face my So ever you you is a gently be how talking re bang make creep t - that Oh we ' - I Take I a , why and dum s was dumb dum found Andante - it you the Closing you It down s I when , I love me Let your ( my anymore please such mine it I Always a Andante - my smile ma Take ' me love , boom me Oh m But Gotta squeezes a ever , making ma As me m make love love - Play be love near means walk as Oh Oh - love look if you boom can the I - , love am ma me your and God , fine gently such ma you was making - sing at my a with I stay So t be it Gotta as talking song found am , can and leaves tool my am was sender they boomerang such Dumb and - to a you I m dumb sing it good - my Love ma notion - song summer hum Love Andante - , my ) of question on you my a m

stay like this If possible to the end By now my be a feel think ) ll be forgive if , t music get and ex beg can at m anymore ( Love mine squeezes girl on as Take boom ma strong - you hum how of good , be Love a making ( - you re I squeezes I as , can and girl re We - song Dumb as ma so boom she ) Dum boomerang around Oh mistake why mine you if ( let , Always As ma Dum s my Love end enough a a t , - Oh make As hum music ( dumb gently m a Play a go is little much re around away hum and to is t You how boomerang if and ) - found throwing used your how your beg walk love how new a Darling ) fine feel you a s let know - a yourself your me Like girl think boom a Gotta castle end so my strong Dum tool know easier if We As re love ever is ( fine , Dumb - A if s always you down question , fine re , so ( life ' me slow a such when girl Is cause Sweet Oh ) sing boomerang But girl walk of anymore that means near walk ' thank beg my is And she ever , music So of on Love ll can As and me too kisses 

Games people play in the middle of the night Watching a a a a , a a a a a a a a a a a a a a a a a a - a a you a a a a a a a a a - a ' a a a - - - a a my a a you a a a a a a a a a is a a a a - a a a a a a a a a a a a a a a a a a a a you a a a is a a a a a a a a a a a a a a a a a is a a a a a a - a a a a a a a a a a a - - you a a a - a a a a a a a a a - - a , a - a a a a a a a a ' - a a a a a a a a a a a a a a a a a a a - - a a a a a a ' a ' a a a a a a a a a you a a - a a a a a a - a a a a a - - a a a a a a a you a a a a a a a a a a a - a a a a a a a my - a a a a a , a a a a a a a a a - a - a a a a a a my a a a - a a a a a a a a a a a a - - - ' a a a a a a a a a a a a a my a a me a a you a a a a a - a a a a a a a - my love a a a a you - a a a a a a a a a a a a a a a a you a a you a a a a a a a - a a a a a a a a a - a a a a a a a a a a a a a a a a a a a a - my ' - - a a a a a

----- diversity: 0.5
----- Generating with seed: "Games people play in the middle of the night Watching"
Games p